In [ ]:
import duckdb
import pandas as pd
import xml.etree.ElementTree as ET

# Función para convertir XML a diccionario
def xml_to_dict(element):
    result = {}
    if element.tag == "goal":
        result["values"] = []  # Inicializa la lista de valores
        for value in element.findall('value'):
            value_dict = xml_to_dict(value)  # Convierte cada valor a dict
            result["values"].append(value_dict)  # Agrega el dict a la lista
    else:
        for child in element:
            # Llamar recursivamente si hay más hijos
            result[child.tag] = xml_to_dict(child) if len(child) > 0 else child.text
    return result

In [ ]:
#tabla paises 
pais = duckdb.sql('''SELECT name AS Nombre FROM enunciado_paises.csv''')
df_pais = pais.df()
df_pais.to_csv('tabla_pais.csv', index=False)

In [ ]:
#tabla jugador
jugador = duckdb.sql('''SELECT player_api_id AS ID, player_name AS Nombre FROM enunciado_jugadores.csv''')
df_jugador = jugador.df()
df_jugador.to_csv('tabla_jugador.csv',index=False)

In [ ]:
#tabla temporada, el id fue creado con pandas
temporada = duckdb.sql('''
                       SELECT DISTINCT season AS Temporada 
                       FROM enunciado_partidos.csv
                       ''')
df_temporada = temporada.df()         # paso el objeto duckdb a df de pandas
df_temporada.to_csv('tabla_temporada.csv', index=False)

In [ ]:
# tabla liga, su correcto ID fue obtenido a pertir del country_id del partido
liga = duckdb.sql('''
                  SELECT DISTINCT l.name AS Nombre_liga, p.name AS Nombre_pais, part.league_id AS ID
                  FROM enunciado_liga.csv AS l
                  INNER JOIN enunciado_paises.csv AS p ON l.country_id = p.id
                  INNER JOIN enunciado_partidos.csv AS part ON l.country_id = part.country_id
                  ''')

df_liga = liga.df()
df_liga.to_csv('tabla_liga.csv', index=False)


In [ ]:
# Tabla Equipos
equipo = duckdb.sql('''
                    SELECT DISTINCT eq.team_api_id AS ID_equipo, eq.team_long_name AS Nombre, par.league_id AS ID_liga
                    FROM enunciado_equipos.csv AS eq
                    INNER JOIN enunciado_partidos.csv AS par 
                    ON eq.team_api_id = par.home_team_api_id
                    ''')

df_equipo = equipo.df()
#EXPLICACION: basicamente hice el join en base a la liga donde ese equipo jugó de local, podria pasar que no hay jugado de local?
# según lo que estuve viendo no, porque me unió a todos los equipos
df_equipo.to_csv('tabla_equipo.csv', index=False)

In [ ]:
# Tabla Equipos DOBLADA para ver si no tiré cualquiera con el inner join
# esta tabla me sirvió para el bloque de código donde veo que todos jugaron de local
equipo = duckdb.sql('''
                    SELECT DISTINCT eq.team_api_id AS ID_equipo, eq.team_long_name AS Nombre
                    FROM enunciado_equipos.csv AS eq
                    ''')

df_equipo = equipo.df()
#EXPLICACION: basicamente hice el join en base a la liga donde ese equipo jugó de local, podria pasar que no hay jugado de local?
# según lo que estuve viendo no, porque me unió a todos los equipos
df_equipo.to_csv('tabla_equipomm.csv', index=False)

In [ ]:
tabla_goles = duckdb.sql('''
                      SELECT DISTINCT match_api_id AS ID_partido, goal AS Goles
                      FROM enunciado_partidos.csv
                      ''')

df_goles = tabla_goles.df().dropna(subset=['Goles'])

# Armo cada gol
counter = 0
final_table = []

row_amount = len(df_goles["Goles"])
print(f"There should be {row_amount} rows.")

final_table = []

id_gol_column = []
id_partido_column = []
id_jugador_column = []
tipo_column = []
subtipo_column = []

 # por cada gol, agarrar la info que nos importa, e insertalo en la tabla.
for i in range(1, 500):
    
    id_partido = int(df_goles["ID_partido"][i])
    set_de_goles = df_goles["Goles"][i]
    
    # Parsear el string XML
    root = ET.fromstring(set_de_goles)
    # Convertir el XML a diccionario. (Si, to_dict estaba ahí nomás para poder iterar)
    goal_dict = xml_to_dict(root)
    
    # tomamos 
    if counter == 0:
        print(goal_dict)
    for gol in goal_dict["values"]:
        if counter < 8: print(f"{counter} - From:\n{gol}")
        id_gol = int(gol["id"])
        if "player1" in gol.keys():
            id_jugador = int(gol["player1"])
        else:
            print(f"The player is missing here:\n{gol}")
            id_jugador = None
        
        if "stats" in gol.keys():
            if ("goals" in gol["stats"].keys()) and ("shoton" in gol["stats"].keys()):
                tipo = "normal"
            elif ("goals" in gol["stats"].keys()) and ("penalty" in gol["stats"].keys()):
                tipo = "penal en juego"
            elif ("penalty" in gol["stats"].keys()):
                tipo = "penal por desempate"
            elif ("owngoals" in gol["stats"].keys()):
                tipo = "contragol"
        else:
            tipo = "-"
        
        if "subtype" in gol.keys():
            subtipo = gol["subtype"]
        else:
            subtipo = "-"
        
        id_gol_column.append(id_gol)
        id_partido_column.append(id_partido)
        id_jugador_column.append(id_jugador)
        tipo_column.append(tipo)
        subtipo_column.append(subtipo)
        
    print(counter)
    counter += 1


intermediary_dict = {}
intermediary_dict["ID_gol"] = id_gol_column
intermediary_dict["ID_partido"] = id_partido_column
intermediary_dict["ID_jugador"] = id_jugador_column
intermediary_dict["Tipo"] = tipo_column
intermediary_dict["Subtipo"] = subtipo_column

tabla_de_goles = pd.DataFrame(intermediary_dict)

tabla_de_goles.to_csv("tabla_goles", index=False)


In [ ]:
# Tabla Partidos
partidos = duckdb.sql('''
                      SELECT DISTINCT match_api_id AS ID_partido, season AS Temporada, date AS Fecha, home_team_api_id AS ID_local, away_team_api_id AS ID_visitante, home_team_goal AS Goles_local, away_team_goal AS Goles_visitante,
                      CASE WHEN home_team_goal > away_team_goal THEN 'Ganado'
                      WHEN away_team_goal > home_team_goal THEN 'Perdido'
                      ELSE 'Empatado'
                      END AS Resultado,
                      FROM enunciado_partidos.csv
                      ''')
partidos = partidos.df()

partidos.to_csv('tabla_partidos.csv', index =False)

In [ ]:
# Consulta para chequear que todos los equipos juegan de local
todos_juegan_local = duckdb.sql('''
                                SELECT ID_equipo
                                FROM tabla_equipomm.csv
                                EXCEPT
                                SELECT ID_local
                                FROM tabla_partidos.csv
                                ''')
si = todos_juegan_local.df()
si

In [ ]:
#Tabla Plantel
# Acá hice un cross join, suponiendo que todos los equipos jugaron todas las temporadas... FALSO
plantel = duckdb.sql('''
                     SELECT DISTINCT Temporada, ID_local AS ID_equipo
                     FROM tabla_partidos.csv 
                     ''')

df_plantel = plantel.df()
df_plantel['ID_plantel'] = range(0,len(df_plantel),1)
df_plantel.to_csv('tabla_plantel.csv', index=False)

In [ ]:
# Conformación plantel
conf_plantel = duckdb.sql('''
                          SELECT DISTINCT ID_plantel, home_player_1 AS ID_jugador
                          FROM enunciado_partidos.csv, tabla_plantel.csv
                          WHERE season = Temporada AND home_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, home_player_2 AS ID_jugador
                          FROM enunciado_partidos.csv, tabla_plantel.csv
                          WHERE season = Temporada AND home_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, home_player_3 AS ID_jugador
                          FROM enunciado_partidos.csv, tabla_plantel.csv
                          WHERE season = Temporada AND home_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, home_player_4 AS ID_jugador
                          FROM enunciado_partidos.csv, tabla_plantel.csv
                          WHERE season = Temporada AND home_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, home_player_5 AS ID_jugador
                          FROM enunciado_partidos.csv, tabla_plantel.csv
                          WHERE season = Temporada AND home_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, home_player_6 AS ID_jugador
                          FROM enunciado_partidos.csv, tabla_plantel.csv
                          WHERE season = Temporada AND home_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, home_player_7 AS ID_jugador
                          FROM enunciado_partidos.csv, tabla_plantel.csv
                          WHERE season = Temporada AND home_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, home_player_8 AS ID_jugador
                          FROM enunciado_partidos.csv, tabla_plantel.csv
                          WHERE season = Temporada AND home_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, home_player_9 AS ID_jugador
                          FROM enunciado_partidos.csv, tabla_plantel.csv
                          WHERE season = Temporada AND home_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, home_player_10 AS ID_jugador
                          FROM enunciado_partidos.csv, tabla_plantel.csv
                          WHERE season = Temporada AND home_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, home_player_11 AS ID_jugador
                          FROM enunciado_partidos.csv, tabla_plantel.csv
                          WHERE season = Temporada AND home_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, away_player_1 AS ID_jugador
                          FROM enunciado_partidos.csv, tabla_plantel.csv
                          WHERE season = Temporada AND away_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, away_player_2 AS ID_jugador
                          FROM enunciado_partidos.csv, tabla_plantel.csv
                          WHERE season = Temporada AND away_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, away_player_3 AS ID_jugador
                          FROM enunciado_partidos.csv, tabla_plantel.csv
                          WHERE season = Temporada AND away_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, away_player_4 AS ID_jugador
                          FROM enunciado_partidos.csv, tabla_plantel.csv
                          WHERE season = Temporada AND away_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, away_player_5 AS ID_jugador
                          FROM enunciado_partidos.csv, tabla_plantel.csv
                          WHERE season = Temporada AND away_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, away_player_6 AS ID_jugador
                          FROM enunciado_partidos.csv, tabla_plantel.csv
                          WHERE season = Temporada AND away_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, away_player_7 AS ID_jugador
                          FROM enunciado_partidos.csv, tabla_plantel.csv
                          WHERE season = Temporada AND away_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, away_player_8 AS ID_jugador
                          FROM enunciado_partidos.csv, tabla_plantel.csv
                          WHERE season = Temporada AND away_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, away_player_9 AS ID_jugador
                          FROM enunciado_partidos.csv, tabla_plantel.csv
                          WHERE season = Temporada AND away_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, away_player_10 AS ID_jugador
                          FROM enunciado_partidos.csv, tabla_plantel.csv
                          WHERE season = Temporada AND away_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, away_player_11 AS ID_jugador
                          FROM enunciado_partidos.csv, tabla_plantel.csv
                          WHERE season = Temporada AND away_team_api_id = ID_equipo
                          ''')



# una vez que hago todos estos simplemente hago un join y listo
conformacion_plantel = conf_plantel.df()
conformacion_plantel = conformacion_plantel.dropna(subset=['ID_jugador']) # acá borré los nulls para poder aplicar la funcion para sacar ese id.0 que aparece en todos los jugadores de partidos
conformacion_plantel['ID_jugador'] = conformacion_plantel['ID_jugador'].apply(lambda x: int(x)) # saco esos .0
conformacion_plantel.to_csv('tabla_conformacion_plantel.csv',index=False)

# agarro la tabla plantel y digo: si en partidos coincide el season con id_temporada Y local con equipo SELECT id_plantel con id_home player

In [ ]:
atributos_jugador = duckdb.sql("""
                               SELECT
                               player_api_id as id_jugador,
                               player_fifa_api_id,
                               date,
                               overall_rating,
                               potential,
                               preferred_foot,
                               crossing,
                               finishing,
                               dribbling,
                               free_kick_accuracy,
                               ball_control,
                               acceleration,
                               sprint_speed,
                               agility,
                               reactions,
                               balance,
                               shot_power,
                               jumping,
                               strength,
                               aggression,
                               interceptions,
                               vision,
                               penalties,
                               marking
                               FROM enunciado_jugadores_atributos.csv
                               """)

df_atributos_jugador = atributos_jugador.df()
df_atributos_jugador.to_csv("tabla_atributos_jugador.csv", index=False)